<a href="https://colab.research.google.com/github/dodonas/CC_Ext_2/blob/main/CLOUD_TECH_EXT_2_PRODUCER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Afeka 65215 Ext 2 - Kafka and Spark Streaming in Colab

# Submitter: Andrey Dodon - 317858298

# Installs and sets

In [17]:
%%capture
# --------- Kafka -----------
!pip install confluent_kafka
!pip install kafka-python
# --------- Spark -----------
!pip install findspark

!apt-get install openjdk-11-jdk-headless -qq > /dev/null

!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"
!wget https://github.com/dodonas/CC_Ext_2/blob/main/RomeoAndJuliet.txt
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

# Imports

In [2]:
from confluent_kafka import Producer
from itertools import islice
import json
import time
import sys
import os
from datetime import datetime
import threading
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1

# Functions

In [7]:
def get_current_time():
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  print("Current Time =", current_time)

In [18]:
def connect_kafka_producer(server):
    _producer = None
    try:
        _producer = Producer({'bootstrap.servers': server, 'auto.offset.reset': 'earliest'})
        print('Connected {}'.format(_producer))
    except Exception as ex:
        print('Exception while connecting Kafka: {}'.format(str(ex)))
    finally:
        return _producer

The following function is called once for each message generated to indicate the delivery result

In [9]:
def delivery_callback(err, msg):
  if err:
    sys.stderr.write('%% Message failed delivery: %s\n' % err)
  else:
    sys.stderr.write('%% Message delivered to %s [%d] @ %d\n' % (msg.topic(), msg.partition(), msg.offset()))

Each time the following function is called it will return the next 100 (or less) lines.

In [10]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Variables

In [19]:
kafka_topic_name = "RomeoAndJuliet"
kafka_bootstrap_servers = 'localhost:9092'

In [11]:
def produceRomeoAndJuliet():
  p = connect_kafka_producer(kafka_bootstrap_servers)
  with open("RomeoAndJuliet.txt") as fp:
    lines = fp.readlines()
    for chunk in chunker(lines, 100):
        for line in chunk:
          try:
            p.produce(kafka_topic_name, line.rstrip(), callback=delivery_callback)
          except BufferError:
            sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))
          p.poll(0)
        time.sleep(2)
  p.flush()

In [16]:
produceRomeoAndJuliet()

Connected <cimpl.Producer object at 0x7f081c832d60>


Streaming output truncated to the last 5000 lines.
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
% Message failed delivery: KafkaError{code=_MSG_TIMED_OUT,